# 컨텐츠 기반 필터링

In [1]:
!pip install scikit-surprise

In [2]:
import numpy as np
import pandas as pd
from surprise import Dataset
data=Dataset.load_builtin('ml-100k',prompt=False)
df=pd.DataFrame(data.raw_ratings,columns=['user-id','movie-id','rating','timestamp'])

In [3]:
df.head()

,user-id,movie-id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [4]:
df.shape

(100000, 4)

### 1.Adjacent Matrix 생성
- 행:사용자
- 열:영화
- 내용:평점

In [5]:
raw_data=np.array(data.raw_ratings,dtype=int)
np.min(raw_data,axis=0)

array([        1,         1,         1, 874724710])

In [6]:
np.max(raw_data,axis=0)

array([      943,      1682,         5, 893286638])

In [7]:
#user id랑 movie id가 0부터 시작하게 만들기
raw_data[:,:2] -=1
raw_data[:5]

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       [      243,        50,         2, 880606923],
       [      165,       345,         1, 886397596]])

In [8]:
#행열개수 알아내기
nrows=df['user-id'].nunique() #행의 갯수 = 사용자 수
ncols=df['movie-id'].nunique() #열의 갯수 = 영화 갯수
nrows,ncols

(943, 1682)

#### 1)본 영화/안본 영화로만 구분, 1,0

In [9]:
adj_matrix=np.zeros([nrows,ncols],int)
for user_id, movie_id,_, _ in raw_data:
    adj_matrix[user_id,movie_id] = 1
adj_matrix[:5]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [10]:
#0번 이용자를 나라고 두고
my_id, my_vector = 0, adj_matrix[0]
#나와 10번 유저 사이의 유사도
np.dot(my_vector,adj_matrix[10]),np.dot(my_vector,adj_matrix[30])

(71, 11)

In [11]:
best_score, best_match_id=0,0

for i in range(1, len(adj_matrix)):
    dot = np.dot(my_vector, adj_matrix[i])
    if dot>best_score:
        best_score, best_match_id= dot, i

best_score, best_match_id

(183, 275)

In [12]:
best_vector = adj_matrix[best_match_id]
my_vector[200:210], best_vector[200:210]

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1]))

#내가 안 본 영화 중에서 best_match 사용자가 본 영화 추천하기

In [13]:
#내가 안 본 영화 중에서 best_match 사용자가 본 영화 추천하기

recommend_list =[]
for i, (my_view,best_match_view) in enumerate(zip(my_vector,best_vector)):
    if my_view == 0 and best_match_view ==1:
        recommend_list.append(i)

len(recommend_list),recommend_list[:10]

(335, [272, 273, 275, 280, 281, 283, 287, 288, 289, 290])

- 평점 점수를 주는 경우

In [14]:
adj_matrix=np.zeros([nrows,ncols],int)
for user_id, movie_id, rating, _ in raw_data:
    adj_matrix[user_id,movie_id] = rating
adj_matrix[:5]

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 3, 0, ..., 0, 0, 0]])

- case1: 유클리드 거리 

In [15]:
best_score, best_match_id=100000,0
my_vector=adj_matrix[0]
for i in range(1, len(adj_matrix)):
    #유클리드 거리 구하기
    euc=np.sqrt(np.sum(np.square(my_vector- adj_matrix[i])))
    if euc < best_score:
        best_score, best_match_id= euc, i

best_score, best_match_id

(55.06359959174482, 737)

In [16]:
#내가 안 본 영화 중에서 best_match 사용자가 본 영화 추천하기
best_vector=adj_matrix[best_match_id]
recommend_list =[]

for i, (my_view,best_match_view) in enumerate(zip(my_vector,best_vector)):
    if my_view == 0 and best_match_view >=1:
        recommend_list.append(i)

len(recommend_list),recommend_list[:10]

(48, [297, 312, 317, 342, 356, 366, 379, 384, 392, 402])

In [17]:
#내가 안 본 영화 중에서 best_match 사용자가 본 '최고 평점'영화 추천하기
best_vector=adj_matrix[best_match_id]
recommend_list =[]

for i, (my_view,best_match_view) in enumerate(zip(my_vector,best_vector)):
    if my_view == 0 and best_match_view ==5: #평점 5 준 것만.
        recommend_list.append(i)

len(recommend_list),recommend_list[:10]

(6, [312, 317, 384, 407, 526, 602])

- 코사인 유사도

In [18]:
def cos_similarity(v1,v2):
    v1_norm = np.sqrt(np.sum(np.square(v1)))
    v2_norm = np.sqrt(np.sum(np.square(v2)))
    return np.dot(v1,v2) / (v1_norm * v2_norm)

In [19]:
#나랑 닮은 사용자
best_score, best_match_id=-1,0
my_vector=adj_matrix[0]


for i in range(1, len(adj_matrix)):
    cos_sim = cos_similarity(my_vector, adj_matrix[i])
    if cos_sim >best_score:
        best_score, best_match_id= cos_sim , i

best_score, best_match_id

(0.569065731527988, 915)

In [20]:
# 내가 보지 않은 영화중에서 best_match 사용자가 본 평점이 좋은 영화 --> 추천
best_vector = adj_matrix[best_match_id]
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_vector)):
    if my_view == 0 and best_match_view >= 5:
        recommend_list.append(i)

len(recommend_list), recommend_list[:10]

(7, [420, 424, 482, 510, 511, 918, 1008])